<a href="https://colab.research.google.com/github/tayfununal/2-Hidden-Layer-Neural-Networks/blob/master/maliyet_fonksiyonu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playground-data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plygdata as pg
import json
import requests

In [ ]:
Datas = json.loads(requests.get("https://raw.githubusercontent.com/tayfununal/2-Hidden-Layer-Neural-Networks/master/produced_XOR_Datas.json").text)

In [ ]:
df = pd.DataFrame(Datas["1"])
df

In [ ]:
X = np.array(df[[0,1]]).T
Y = np.array(df[[2]]).T

In [ ]:
print("Shape of X_train:", X.shape,
      "\nShape of y_train:", Y.shape)

In [ ]:
def initialization_parameters(x, y, num_node, random_seeds_one, random_seeds_two):
    np.random.seed(random_seeds_one)
    W1 = np.random.randn(num_node * x.shape[0]).reshape(num_node, x.shape[0])
    b1 = np.zeros((num_node, 1))

    np.random.seed(random_seeds_two)
    W2 = np.random.randn(y.shape[0], num_node)
    b2 = np.zeros((y.shape[0], 1))

    assert W1.shape == (num_node, x.shape[0])
    assert b1.shape == (num_node, 1)

    assert W2.shape == (y.shape[0], num_node)
    assert b2.shape == (y.shape[0], 1)

    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2}
    return parameters

def sigmoid(x):
  return 1/(1+np.exp(-x))

def relu(x):
  return np.maximum(0,x)

def reluDerivative(x):
  x[x<=0] = 0
  x[x>0] = 1
  return x

def forward_prop(x,parameters):
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  
  Z1 = np.dot(W1, x) + b1
  A1 = relu(Z1)
  Z2 = np.dot(W2, A1) + b2
  A2 = sigmoid(Z2)

  assert (A2.shape == (1, x.shape[1]))
  cache = {
      'Z1' : Z1,
      'A1' : A1,
      'Z2' : Z2,
      'A2' : A2
  }
  return A2, cache

def backward_prop(x, y, parameters, cache, learning_rate = 0.1):
  m = y.shape[1]
  
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']

  A1 = cache['A1']
  A2 = cache['A2']
  
  # Calculations of backward propagation: dW1, db1, dW2, db2
  dZ2 = A2 - y
  dW2 = (1 / m) * np.dot(dZ2, A1.T)
  db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True)

  dZ1 = np.multiply(np.dot(W2.T, dZ2), reluDerivative(A1))
  dW1 = (1 / m) * np.dot(dZ1, x.T)
  db1 = (1 / m) * np.sum(dZ1 , axis = 1, keepdims = True)

  # Updating parameters
  W1 = W1 - learning_rate * dW1
  b1 = b1 - learning_rate * db1
  W2 = W2 - learning_rate * dW2
  b2 = b2 - learning_rate * db2
  
  parameters = {'W1':W1,
                'b1':b1,
                'W2':W2,
                'b2':b2}
  return parameters

def cross_entropy_cost(y, A2):
  m = y.shape[1]
  
  cross_entropy = np.multiply(np.log(A2 + 1e-15), y) + np.multiply((1 - y), np.log(1 - A2 + 1e-15))
  cost = (- 1.0 / m) * np.sum(cross_entropy)

  # Squeezing to avoid unnecessary dimensions 
  cost = np.squeeze(cost) 
  return cost  

In [ ]:
def nn_model(x, y, parameters, number_of_iter = 1000):
  A2, cache = forward_prop(x, parameters)
  cost = cross_entropy_cost(y, A2)
  
  for i in range(1,number_of_iter):
    parameters = backward_prop(x, y, parameters, cache, learning_rate=0.6)
    A2, cache = forward_prop(x, parameters)
    cost = cross_entropy_cost(y, A2)

  return cost, parameters

In [ ]:
np.random.seed(65468)
random_seeds = np.zeros((100,2))
for i in range(0,100):
  for j in range(0,2):
    random_seeds[i,j] = np.random.randint(1,100001)

In [ ]:
from collections import Counter
each_of_data_Result = dict()

for n in range(1,2):  #Toplam 100 tane veri grubu olduğundan 100 kere çalıçmasını sağlamak için 101 e kadar saydırmalıyız.
  df = pd.DataFrame(Datas["{}".format(n)])
  X = np.array(df[[0,1]]).T
  Y = np.array(df[[2]]).T
  print(n)

  average_aHundred_cost = Counter(dict())
  for i in range(0,100):             #100 farklı başlangıç noktası
    average_twenty_cost = dict()
    for number_of_neuron in range(1,21):
      parameters = initialization_parameters(X, Y, number_of_neuron, int(random_seeds[i,0]), int(random_seeds[i,1]))
      cost, parameters = nn_model(X , Y, parameters, number_of_iter = 1000)
      average_twenty_cost['{}'.format(number_of_neuron)] = cost * (1/100)   #100 tane farklı başlangıç noktası ile modeli eğittiğimiz için 100'e bölüyoruz.
    average_aHundred_cost += Counter(average_twenty_cost)
  each_of_data_Result["{}".format(n)] = dict(average_aHundred_cost)

print(each_of_data_Result)

In [ ]:
cost_value = []
for i in range(1,21):
 parameters = initialization_parameters(X, Y, i)
 cost, parameters = nn_model(X, Y, parameters, number_of_iter = 1000)
 cost_value.append("{}".format(cost))


In [ ]:
parameters

In [ ]:
cost_value

In [ ]:
y = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95]
t =[0,4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,89,94]
maliyet_cost = [float(maliyet_cost[i]) for i in t]
baski_maliyet = [float(baski_maliyet[i]) for i in y]

x = [i for i in t]


In [ ]:
plt.plot(x,maliyet_cost,color="blue")
plt.plot(x,baski_maliyet,color="red")